# Graphs from Real-World Data, Milestone 1

A good place to start with a knowledge graph is to look at the source data and learn more about it and what it contains. For our use case we're going to work with NYC public data - tax records, deeds, and building permits (see the liveProject for more background on each of these data resources). Note that the source files are huge (the original tax files are 3.3 GB and have 139 columns!), so I've reduced the size and scope to make this more manageable. The project details all links to the source files if you want to explore more by yourself.

You're going to prepare two sets of files - one for a subset of Manhattan zipcodes and one for all of Manhattan. At the end of each data section you are asked questions about the data.

In [19]:
import os 

DATA_DIR = "./data"
INPUT_TAX_DATA_CSV = os.path.join(DATA_DIR, "tax.csv")
INPUT_DEED_DATA_CSV = os.path.join(DATA_DIR, "deed.csv")
INPUT_PERMIT_DATA_CSV = os.path.join(DATA_DIR, "permit.csv")

PERMIT_COLUMNS = [
    'BOROUGH', 
    'House #', 
    'Street Name', 
    'Zip Code',
    'Job #',
    'Block', 
    'Lot', 
    'Issuance Date', 
    "Owner's Business Type", 
    "Owner's Business Name", 
    "Owner's First Name", 
    "Owner's Last Name", 
    "Owner's House #", 
    "Owner's House Street Name", 
    "Owner's House City", 
    "Owner's House State", 
    "Owner's House Zip Code"
]

FINAL_SMALL_TAX_DATA_CSV = os.path.join(DATA_DIR, "tax_small_final.csv")
FINAL_SMALL_DEED_DATA_CSV = os.path.join(DATA_DIR, "deed_small_final.csv")
FINAL_SMALL_PERMIT_DATA_CSV = os.path.join(DATA_DIR, "permit_small_final.csv")

FINAL_LARGE_TAX_DATA_CSV = os.path.join(DATA_DIR, "tax_final.csv")
FINAL_LARGE_DEED_DATA_CSV = os.path.join(DATA_DIR, "deed_final.csv")
FINAL_LARGE_PERMIT_DATA_CSV = os.path.join(DATA_DIR, "permit_final.csv")

### 1. Load and filter tax records

The tax assessor file contains multiple years of records for all of the properties in Manhattan. This is going to be more than we need for this milestone, so we're going to filter the data:

* Filter to three zipcodes (ZIP_CODE) - 10001, 10016, 10118
* Filter to valid street addresses (i.e. non-null values of HOUSENUM_LO)

In [20]:
import polars as pl 
pl.Config.with_columns_kwargs = True

tax_small = (pl
    .read_csv(INPUT_TAX_DATA_CSV, ignore_errors=True)
    .filter(pl.col("ZIP_CODE").is_in([10001, 10016, 10118]))
    .with_columns(
        BORO = pl.col("BORO").cast(str), 
        BLOCK = pl.col("BLOCK").cast(str).str.zfill(5),
        LOT = pl.col("LOT").cast(str).str.zfill(4),
    )
    .filter(pl.col("HOUSENUM_LO").is_not_null())
    .filter(pl.col("HOUSENUM_HI").is_not_null())
    .with_column(
        pl.when(
            pl.col("APTNO").is_null()).then(pl.lit("")).otherwise(pl.col("APTNO")).alias("APTNO")
    )  
    .with_columns(
        BBL = pl.col("BORO") + pl.col("BLOCK") + pl.col("LOT"), 
        ADDRESS = pl.col("HOUSENUM_LO").cast(str) + " " + pl.col("STREET_NAME") + " " + pl.col("APTNO") + " NYC NY " + pl.col("ZIP_CODE").cast(str))
    .select([
        pl.col("BBL").alias("bbl"), 
        pl.col("OWNER").alias("tax_owner_name"),
        pl.col("ADDRESS").alias("tax_property_address")
     ] 
    )
)
tax_small

bbl,tax_owner_name,tax_property_address
str,str,str
"""1005591001""","""WANG, MICHEL S...","""137 EAST 37 ST..."
"""1005597502""","""UNAVAILABLE OW...","""137 EAST 37 ST..."
"""1007227501""","""UNAVAILABLE OW...","""420 WEST 25 ST..."
"""1008037502""","""NEWMARK,KNIGHT...","""301 7 AVENUE ..."
"""1006980032""","""26-10 CORP""","""279 10 AVENUE ..."
"""1007017502""","""UNAVAILABLE OW...","""500 WEST 30 ST..."
"""1006620010""","""NYS DOT""","""163 12 AVENUE ..."
"""1006620013""","""NYS DOT""","""199 12 AVENUE ..."
"""1006700001""","""UNITED STATES ...","""201 11 AVENUE ..."


We're going to have separate entities for properties and addresses, so we need to add new columns that we can extract when loading the file into Neo4j. We'll need two new columns:

* BBL - as mentioned in the project, NYC organizes tax lots using the BOROUGH-BLOCK-LOT (BBL) system, where BOROUGH is a 1 digit number (BOROUGH = 1 is Manhattan), BLOCK is 5 digits, and LOT is 4 digits. We need to concat these columns into a single identifier.
  * For example, the Empire State Building is in Manhattan (borough = 1), block is 835 (zero-padded to 00835), and lot is 0041 (zero-padded to 0041), giving a BBL of 1008350041. 
* Address - many buildings have a range of street numbers, which is why the tax data contains HOUSENUM_LO (the lowest number in the range) and HOUSENUM_HI (the highest number in the range). For this exercise we're going to only use the low number for consistency. We're going to create a new address column by concatenating fields into this pattern: "\<HOUSENUM_LO\> \<STREET_NAME\> \<APTNO\> NYC NY \<ZIP_CODE\>"
  * Note that APTNO (apartment number) can be null; please filter out null values from the address string. 

In [10]:
tax_small.write_csv(FINAL_SMALL_TAX_DATA_CSV)

### 2. Load and filter property deeds

We're going to look at property deeds (i.e. transfer of real estate from one party to another). Normally this is split between five different datasets - for simplicity we've combined them into a single set of data.

We're going to repeat what we did before:

* Add a column for the property's BBL
* Add a column for Address - note that the document has two columns, one for property address and one for the new owner's address. Since we already have the property address, we're going to capture the owner's address (ADDRESS 1, ADDRESS 2, CITY, STATE, ZIP)
* Filter to the properties in our target zip codes (10001, 10016, 10118)

In [12]:
deed_small = (pl
           .read_csv(INPUT_DEED_DATA_CSV, ignore_errors=True)
           .unique()
           .filter(pl.col("ZIP").is_in([10001, 10016, 10118]))
           .with_columns(
               BOROUGH = pl.col("BOROUGH").cast(str),
               BLOCK = pl.col("BLOCK").cast(str).str.zfill(5),
               LOT = pl.col("LOT").cast(str).str.zfill(4),
           )
           .with_column(
               pl.when(pl.col("ADDRESS 2").is_null()).then(pl.lit(" ")).otherwise(pl.col("ADDRESS 2")).alias("ADDRESS 2") 
           )
           .with_columns(
               BBL = pl.col("BOROUGH") + pl.col("BLOCK") + pl.col("LOT"), 
               OWNER_ADDRESS = pl.col("ADDRESS 1") + " " + pl.col("ADDRESS 2") + pl.col("CITY") + " " + pl.col("STATE") + " " + pl.col("ZIP").cast(str),
           )
           .select([
               pl.col("BBL").alias("bbl"), 
               pl.col("DOCUMENT ID").alias("deed_id"), 
               pl.col("DOC. DATE").alias("deed_date"),
               pl.col("NAME").alias("deed_owner_name"),
               pl.col("OWNER_ADDRESS").alias("deed_owner_address")
            ]
        )
)


In [13]:
deed_small

bbl,deed_id,deed_date,deed_owner_name,deed_owner_address
str,i64,str,str,str
"""1008942081""",2015071700095001,"""07/07/2015""","""CYRIAC, GEORGE...","""325 LEXINGTON ..."
"""1008942081""",2015071700095001,"""07/07/2015""","""CYRIAC, CHACKU...","""325 LEXINGTON ..."
"""1008591119""",2015081101135002,"""08/05/2015""","""11 EAST 29TH S...","""240 FIFTH AVEN..."
"""1012121103""",2015072300008002,"""07/21/2015""","""ALLEN A. MEYER...","""C/O SCOTT SEGA..."
"""1012121103""",2015072300008002,"""07/21/2015""","""ALLEN A. MEYER...","""C/O SCOTT SEGA..."
"""1006981032""",2015081001098002,"""08/04/2015""","""HARIRI, MOJGAN...","""39 WEST 29TH S..."
"""1006981032""",2015081001098002,"""08/04/2015""","""HARIRI, GISUE""","""39 WEST 29TH S..."
"""1012631048""",2015072201289001,"""07/13/2015""","""WUNG, HAN-YUN""","""50 EAST 28TH S..."
"""1008621329""",2015081400392002,"""08/10/2015""","""ANASIS LLC""","""325 FIFTH AVEN..."


In [14]:
deed_small.write_csv(FINAL_SMALL_DEED_DATA_CSV)

### 3. Load and filter building permits

Our last dataset - these are approved building permits that allow the property owner to make structural changes (e.g. add a window, install a new plumbing system, etc).

This is another one of those datasets with more data than we need - it captures information about the property owner, but also includes information about type of work being done and the person requesting the permit. This would be useful if we wanted to learn more about what type of property work is done in NYC and who is doing the work. 

* Add a column for the BBL
* Add a column for the owner's address
* Filter to the properties in our target zip codes (10001, 10016, 10118)

Note that we learn two things about the owner - the name of their business and their name! Let's also create a column for that, with convention "\<Owner's Last Name\>, \<Owner's First Name\>".

In [31]:
permit_small = (pl
           .read_csv(INPUT_PERMIT_DATA_CSV, ignore_errors=True)
           .unique()
           .filter(pl.col("Owner's House Zip Code").is_in([10001, 10016, 10118]))
           .with_columns(
               BOROUGH = pl.col("BOROUGH").cast(str),
               Block = pl.col("Block").cast(str).str.zfill(5),
               Lot = pl.col("Lot").cast(str).str.zfill(4),
           )
           .with_columns(
               BBL = pl.lit("1") + pl.col("Block") + pl.col("Lot"),
               OWNER_FULL_NAME = pl.col("Owner's First Name") + " " + pl.col("Owner's Last Name"), 
               OWNER_ADDRESS = pl.col("Owner's House #") + pl.col("Owner's House Street Name") + pl.col("Owner's House City") + pl.col("Owner's House State") + pl.col("Owner's House Zip Code")
           )
           .select([
               pl.col("BBL").alias("bbl"),
               pl.col("Issuance Date").alias("permit_date"),
               pl.col("OWNER_FULL_NAME").alias("permit_owner_name"),
               pl.col("OWNER_ADDRESS").alias("permit_owner_address"), 
               pl.col("Owner's Business Name").alias("permit_owner_business_name"),
               pl.col("Owner's Phone #").alias("permit_owner_phone"),
           ])
)


In [32]:
permit_small.write_csv(FINAL_SMALL_PERMIT_DATA_CSV)

### 4. Prepare the full set of data

Now that we're comfortable working with our records, we're going to create a second set of files that covers all of Manhattan. 

* Repeat the tax assessor data pull in Step 1, but this time don't filter on zip code
* Same for deeds and permits

In [31]:
tax = (
    pl.read_csv(INPUT_TAX_DATA_CSV, ignore_errors=True)
    .with_columns(
        BORO = pl.col("BORO").cast(str), 
        BLOCK = pl.col("BLOCK").cast(str).str.zfill(5),
        LOT = pl.col("LOT").cast(str).str.zfill(4),
    )
    .filter(pl.col("HOUSENUM_LO").is_not_null())
    .filter(pl.col("HOUSENUM_HI").is_not_null())
    .with_column(
        pl.when(
            pl.col("APTNO").is_null()).then(pl.lit("")).otherwise(pl.col("APTNO")).alias("APTNO")
    )   
    .with_columns(
        BBL = pl.col("BORO") + pl.col("BLOCK") + pl.col("LOT"), 
        ADDRESS = pl.col("HOUSENUM_LO").cast(str) + " " + pl.col("STREET_NAME") + " " + pl.col("APTNO") + " NYC NY " + pl.col("ZIP_CODE").cast(str)
)) 

tax.write_csv(FINAL_LARGE_TAX_DATA_CSV)

In [25]:
deed = (pl
           .read_csv(INPUT_DEED_DATA_CSV, ignore_errors=True)
           .unique()
           .with_column(
               pl.when(pl.col("ADDRESS 2").is_null()).then(pl.lit(" ")).otherwise(pl.col("ADDRESS 2")).alias("ADDRESS 2") 
           )
           .with_columns(
               BOROUGH = pl.col("BOROUGH").cast(str),
               BLOCK = pl.col("BLOCK").cast(str).str.zfill(5),
               LOT = pl.col("LOT").cast(str).str.zfill(4),
               OWNER_ADDRESS = pl.col("ADDRESS 1") + " " + pl.col("ADDRESS 2") + pl.col("CITY") + " " + pl.col("STATE") + " " + pl.col("ZIP").cast(str)
           ) 
)

deed.write_csv(FINAL_LARGE_DEED_DATA_CSV)

In [31]:
tax_small.filter(pl.col("tax_owner").str.contains("LLC"))

bbl,tax_owner,tax_address
str,str,str
"""1006700070""","""BULGROUP COLOR...","""231 11 AVENUE ..."
"""1006720001""","""RXR SL OWNER L...","""601 WEST 26 ST..."
"""1006750001""","""260 TWELFTH AV...","""260 12 AVENUE ..."
"""1006750012""","""WEST SIDE 11TH...","""613 WEST 29 ST..."
"""1006960038""","""500 W25TH OWNE...","""255 10 AVENUE ..."
"""1006960042""","""510 WEST 25TH ...","""510 WEST 25 ST..."
"""1006960049""","""530 WEST 25TH ...","""516 WEST 25 ST..."
"""1006961101""","""SAWYERVILLE MA...","""245 10 AVENUE ..."
"""1006961102""","""ART SPACE 245,...","""245 10 AVENUE ..."
